In [69]:
# Combine all data from P1, P2 into final CCRI layer

In [70]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [71]:
import os
import pandas as pd
import geopandas as gpd
import glob
import numpy as np

In [72]:
# Query data dir (avoiding hard-coding paths when working between users)
#/content/drive/MyDrive/CCRI/ccri_repo/data
data_dir = '/content/drive/MyDrive/CCRI/ccri_repo/data'
print(data_dir)

/content/drive/MyDrive/CCRI/ccri_repo/data


In [73]:
# Check that data dir was input correctly
data_dir

'/content/drive/MyDrive/CCRI/ccri_repo/data'

In [74]:
# Read in all the data we need
# -- exposure files
p1_exposure_file = pd.read_csv('{}/CCRI_results_misc/Merged_Exposure_Data.csv'.format(data_dir))
p2_exposure_file = pd.read_csv('{}/CCRI_results_misc/P2_Merged_Normalized_avg.csv'.format(data_dir))
p1p2_scores = pd.read_csv('{}/CCRI_results_misc/p1_p2_avg_ccri.csv'.format(data_dir))

# -- attribute files
wb_income = pd.read_csv('{}/misc/WB_INCOME.csv'.format(data_dir))
unicef_ro = pd.read_csv('{}/misc/UNICEF_PROG_REG_GLOBAL.csv'.format(data_dir))

# -- population files
childpop = pd.read_csv('{}/CCRI_results_misc/child_pop_sum_adm0.csv'.format(data_dir))
#worldpop = pd.read_csv('{}/Misc/World_Population_ByAOI_adm0.csv'.format(data_dir))

# -- boundary file
adm0 = gpd.read_file('{}/misc/adm0_boundaries_simple.geojson'.format(data_dir))

# -- fragile codes
fragile = pd.read_csv('{}/misc/List of fragile context (2025).csv'.format(data_dir))

# -- component vals
p1_components = pd.read_csv('{}/CCRI_results_misc/p1_group_mean.csv'.format(data_dir))
p2_components = pd.read_csv('{}/CCRI_results_misc/p2_group_mean.csv'.format(data_dir))

In [75]:
p1_components

,iso3,adm0_name,river_flood_gmean,coastal_flood_gmean,storm_gmean,drought_gmean,heatwave_gmean,extreme_heat_gmean,fire_gmean,sand_dust_gmean,air_pollution_gmean,malaria_gmean
0,ALB,Albania,3.827555,5.170350,1.000000,2.912773,5.362889,1.959139,2.977234,1.635030,8.136308,1.000000
1,AFG,Afghanistan,4.032200,1.000000,1.000000,6.436550,6.924245,6.322385,2.719319,8.719157,9.478642,6.414645
2,AND,Andorra,1.000000,1.000000,1.000000,3.311803,3.732753,1.000000,1.000000,1.000000,6.431805,1.000000
3,AGO,Angola,3.238935,2.263277,1.000000,8.114257,4.660324,4.288994,7.391370,3.891533,9.439303,3.130074
4,ARM,Armenia,2.417735,1.000000,1.000000,4.838595,4.271290,1.905620,1.914405,2.162995,8.133400,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
190,ZAF,South Africa,2.465527,2.123107,4.124629,4.812211,7.306105,3.650679,6.545734,3.143219,9.191912,2.092383
191,ZWE,Zimbabwe,2.320742,1.000000,2.532558,6.529357,5.579227,4.808847,6.746355,2.595257,9.139155,3.080333
192,ZMB,Zambia,2.735303,1.000000,1.000000,5.101648,4.716822,4.213679,7.577010,2.115463,9.169472,3.095609
193,RUS,Russian Federation,3.997851,2.400885,2.761276,6.907815,8.430356,2.637057,3.670103,2.641908,9.428427,1.000000


In [76]:
# Renaming some columns
p1_exposure_file.columns = [col.replace('_absolute', '_abs_norm') if '_absolute' in col else col for col in p1_exposure_file.columns]
p1_exposure_file.columns = [col.replace('_relative', '_rel_norm') if '_relative' in col else col for col in p1_exposure_file.columns]

In [77]:
# Merge P exposures by ISO3
merged_P = (p1_exposure_file.merge(p2_exposure_file, on='iso3', how='left'))
all_P = (merged_P.merge(p1p2_scores, on='iso3', how='left'))

In [78]:
all_P = all_P.drop(columns=['P2_arithmetic_avg_y', 'rank_reverse_x'])
all_P = all_P.rename(columns={'P2_arithmetic_avg_x': 'P2_arithmetic_avg', 'rank_reverse_y': 'rank_reverse'})

In [79]:
# Add WB income
wb_income = wb_income[['Region_Code', 'ISO3Code']]
wb_income['Region_Code'] = wb_income['Region_Code'].str.extract(r'WB_(.*)')
df = (all_P.merge(wb_income, left_on='iso3', right_on='ISO3Code', how='left').drop('ISO3Code', axis=1).rename(columns={'Region_Code': 'wb_income'}))

<ipython-input-79-3728206169>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wb_income['Region_Code'] = wb_income['Region_Code'].str.extract(r'WB_(.*)')


In [80]:
# Add Regional Office
unicef_ro = unicef_ro[['Region_Code','ISO3Code']]
unicef_ro['Region_Code'] = unicef_ro['Region_Code'].str.extract(r'UNICEF_(.*)')
df = (df.merge(unicef_ro, left_on='iso3', right_on='ISO3Code', how='left').drop('ISO3Code', axis=1).rename(columns={'Region_Code': 'unicef_ro'}))

<ipython-input-80-465392791>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unicef_ro['Region_Code'] = unicef_ro['Region_Code'].str.extract(r'UNICEF_(.*)')


In [81]:
# Add population data
# -- Take the childpop data from the geojson of p1p2 avg
gdf = gpd.read_file('{}/CCRI_results_misc/p1_p2_avg_ccri.gpkg'.format(data_dir))
df_grouped = gdf[['ISO3', 'child_population_total', 'population_total']].groupby('ISO3', as_index=False).mean()
df_w_childpop = (df.merge(df_grouped, left_on=['iso3'], right_on=['ISO3'], how='left').rename(columns={'child_population_total': 'u18_pop'})).drop(columns=['ISO3'])

#df_w_childpop = (df.merge(childpop, left_on='iso3', right_on='ISO3', how='left').rename(columns={'child_population': 'u18_pop'})).drop(columns=['child_population_gpw'])
#df_w_childpop['u18_pop'] = df_w_childpop['u18_pop'].astype(int)

In [82]:
df_w_childpop = df_w_childpop.rename(columns={'iso3':'ISO3'})

In [83]:
'''
# Using World_Population_ByAOI_adm0.csv file -> removing for now to use the population data from the exposure files to keep consistent
df_w_allpop = (df_w_childpop.merge(worldpop, left_on='name', right_on='adm0_name', how='left').rename(columns={'sum': 'total_pop'})).drop(columns=['iso3','adm0_name'])

# Set population data as integer
df_w_allpop['u18_pop'] = df_w_allpop['u18_pop'].astype(int)
df_w_allpop['total_pop'] = df_w_allpop['total_pop'].astype(int)
'''

"\n# Using World_Population_ByAOI_adm0.csv file -> removing for now to use the population data from the exposure files to keep consistent\ndf_w_allpop = (df_w_childpop.merge(worldpop, left_on='name', right_on='adm0_name', how='left').rename(columns={'sum': 'total_pop'})).drop(columns=['iso3','adm0_name'])\n\n# Set population data as integer\ndf_w_allpop['u18_pop'] = df_w_allpop['u18_pop'].astype(int)\ndf_w_allpop['total_pop'] = df_w_allpop['total_pop'].astype(int)\n"

In [84]:
# Using simplified boundaries for geometry
adm0 = adm0[['ISO3', 'name', 'ucode','uuid','geometry','type']]
df_combined = (df_w_childpop.merge(adm0, left_on=['ISO3', 'adm0_name'], right_on=['ISO3', 'name'], how='left'))

In [85]:
# Grabbing actual exposure numbers
# Define file paths
exposure_path = "{}/p1_exposure".format(data_dir)

# Get all CSV files for exposure
exposure_files = glob.glob(os.path.join(exposure_path, "*.csv"))

# Initialize empty list for processed data
exposure_data_list = []

### **Process Each File in One Loop**
for file in exposure_files:
    df = pd.read_csv(file)  # Read full file to check available columns
    filename_only = os.path.basename(file)
    hazard_name = '_'.join(filename_only.split('_')[:2])  # Extract hazard name

    # Ensure required columns exist
    required_cols = {'iso3', 'adm0_name', 'child_population_exposed'}
    if not required_cols.issubset(df.columns):
        print(f"Skipping {file}: Missing columns {required_cols - set(df.columns)}")
        continue  # Skip if required columns are missing

    df.dropna(subset=['child_population_exposed'], inplace=True)

    # Compute relative exposure (%)
    df['{}_rel'.format(hazard_name)] = np.where(
        (df['child_population_total'] > 0) & (~df['child_population_total'].isna()),
        (df['child_population_exposed'] / df['child_population_total']) * 100,
        0
    )

    # Rename to hazard
    df = df.rename(columns={'child_population_exposed': '{}_abs'.format(hazard_name)})
    df = df.drop(columns=['child_population_total', 'population_total'])

    exposure_data_list.append(df)

In [86]:
merged_exposure_df = pd.concat(exposure_data_list, axis=1).drop_duplicates(subset=['iso3', 'adm0_name'])
# Ensure no duplicate columns before merging
merged_exposure_df = merged_exposure_df.loc[:, ~merged_exposure_df.columns.duplicated()]

In [87]:
merged_exposure_df = merged_exposure_df.drop(columns=['type'])

In [88]:
df_combined = (df_combined.merge(merged_exposure_df, left_on=['ISO3', 'adm0_name'], right_on=['iso3', 'adm0_name'], how='left'))

In [89]:
df_combined = df_combined.rename(columns={'population_total': 'total_pop'})

In [90]:
# --- Adding P2 data
vul_path = "{}/p2_vulnerability".format(data_dir)
total_population_file = "{}/p1_exposure/agricultural_drought_fao_1984-2023_exposure_adm0.csv".format(data_dir)

# Load total child population data
total_pop_df = pd.read_csv(total_population_file, usecols=['iso3', 'adm0_name', 'child_population_total'])
total_pop_df = total_pop_df.rename(columns={'iso3': 'ISO3'})

# Ensure unique ISO3-name pairs before merging
total_pop_df = total_pop_df.groupby(['ISO3', 'adm0_name'], as_index=False).agg({'child_population_total': 'mean'})

# Get all CSV files for exposure
p2_vul_files = glob.glob(os.path.join(vul_path, "*.csv"))

# Initialize empty list for processed data
vul_data_list = []

for file in p2_vul_files:
    df = pd.read_csv(file)  # Read full file to check available columns
    if 'iso3' not in df.columns or 'value' not in df.columns:
        continue  # Skip files missing required columns
    # Normalize 'value' column
    df = df[['iso3', 'value']].dropna()
    df = df.rename(columns={'iso3': 'ISO3'}, errors='ignore')
    filename_only = os.path.basename(file)
    hazard_name = '_'.join(filename_only.split('.csv')[:1])  # Extract hazard name
    if hazard_name == 'P2_Child_Mortality':
        continue

    # merge with population data
    df = (df.merge(total_pop_df, on='ISO3', how='left'))

    # rename to hazard and norm
    df['{}'.format(hazard_name)] = np.where(
        (df['child_population_total'] > 0) & (~df['child_population_total'].isna()),
        (df['value']),
        0
    )

    # Rename for relative
    df = df.drop(columns=['child_population_total', 'time_period', 'data_source', 'value'], errors='ignore')

    vul_data_list.append(df)

In [91]:
for i in range(len(vul_data_list)):
    df_combined = (df_combined.merge(vul_data_list[i], on=['ISO3', 'adm0_name'], how='left'))

In [92]:
# Add fragile
fragile['fragile'] = 'fragile'
df_combined = (df_combined.merge(fragile[['ISO3','fragile']], on=['ISO3'], how='left'))

In [93]:
df_combined = df_combined.drop(columns=['iso3'])

In [94]:
# Add components
# Rename
p1_components = p1_components.rename(columns={'river_flood_gmean': 'P1_rfl', 'coastal_flood_gmean': 'P1_cfl',
                                             'storm_gmean': 'P1_ts', 'drought_gmean': 'P1_dr', 'heatwave_gmean': 'P1_hw', 'extreme_heat_gmean': 'P1_ext',
                                             'fire_gmean': 'P1_fr', 'sand_dust_gmean': 'P1_sds', 'air_pollution_gmean': 'P1_pm25',
                                             'malaria_gmean': 'P1_mal'})

p2_components = p2_components.rename(columns={'health': 'P2_hea', 'nutrition': 'P2_nut', 'education': 'P2_edu', 'wash' : 'P2_wash',
                                             'protection': 'P2_pro', 'poverty': 'P2_pov', 'survival': 'P2_sur'})

In [95]:
df_combined = (df_combined.merge(p1_components, left_on=['ISO3', 'adm0_name'], right_on=['iso3', 'adm0_name'], how='left'))

In [96]:
df_combined = (df_combined.merge(p2_components, left_on=['ISO3'], right_on=['iso3'], how='left'))

In [97]:
df_combined = df_combined.drop(columns=['adm0_name'])

In [98]:
# Remove any columns we don't want and rearrange as well
# -- Dropping
# Drop min, max
df_combined = df_combined.drop(df_combined.filter(regex='max').columns, axis=1)
df_combined = df_combined.drop(df_combined.filter(regex='min').columns, axis=1)

# -- Renaming
df_combined = df_combined.rename(columns={'name': 'adm_name', 'P1_P2_geometric_avg': 'ccri', 'ISO3':'iso3'})

In [99]:
# Set to 2 decimal places
for col in df_combined.columns:
    if type(df_combined['{}'.format(col)].iloc[0]) != str:
        if col in ['wb_income', 'unicef_ro', 'geometry', 'fragile']:
            continue
        else:
            df_combined[col] = df_combined[col].round(2)

In [100]:
# Rename normalized to _norm
df_combined.columns = [col.replace('_normalized', '_norm') if 'normalized' in col else col for col in df_combined.columns]

In [101]:
# Renaming the hazards
# P1 hazards
df_combined.columns = [col.replace('river_flood', 'rfl') if 'river_flood' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('coastal_flood', 'cfl') if 'coastal_flood' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('tropical_storm', 'ts') if 'tropical_storm' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('agricultural_drought', 'agdr') if 'agricultural_drought' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('drought_spei', 'metdr_spei') if 'drought_spei' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('drought_spi', 'metdr_spi') if 'drought_spi' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('heatwave_frequency', 'hw_fre') if 'heatwave_frequency' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('heatwave_severity', 'hw_sev') if 'heatwave_severity' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('heatwave_duration', 'hw_dur') if 'heatwave_duration' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('extreme_heat', 'ext') if 'extreme_heat' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('fire_frequency', 'fr_fre') if 'fire_frequency' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('fire_FRP', 'fr_int') if 'fire_FRP' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('sand_dust', 'sds') if 'sand_dust' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('vectorborne_malariapv', 'mal_pv') if 'vectorborne_malariapv' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('vectorborne_malariapf', 'mal_pf') if 'vectorborne_malariapf' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('air_pollution', 'pm25') if 'air_pollution' in col else col for col in df_combined.columns]

# P2 hazards
df_combined.columns = [col.replace('P2_Immunization_DTP1', 'hea_dtp1') if 'P2_Immunization_DTP1' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_Immunization_DTP3', 'hea_dtp3') if 'P2_Immunization_DTP3' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_Birth_Attendant_Y15T19', 'hea_skat') if 'P2_Birth_Attendant_Y15T19' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_electricity_access', 'hea_elec') if 'P2_electricity_access' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_Nutrition_Stunting_Modeled', 'nut_stu') if 'P2_Nutrition_Stunting_Modeled' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_food_poverty', 'nut_fpov') if 'P2_food_poverty' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_WASH_Drinking_Water', 'wash_wat') if 'P2_WASH_Drinking_Water' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_WASH_Drinking_Sanitation', 'wash_san') if 'P2_WASH_Drinking_Sanitation' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_WASH_Sanitation', 'wash_san') if 'P2_WASH_Sanitation' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_basic_hygiene', 'wash_hyg') if 'P2_basic_hygiene' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_LSCED', 'edu_lsos') if 'P2_LSCED' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_ED_CR_L2', 'edu_lscr') if 'P2_ED_CR_L2' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_Learning_Poverty', 'edu_lpov') if 'P2_Learning_Poverty' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_PT_Labor', 'pro_lab') if 'P2_PT_Labor' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_Child_Marriage', 'pro_mar') if 'P2_Child_Marriage' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_Child_poverty', 'prov_md') if 'P2_Child_poverty' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_Social_Protection', 'prov_u5sp') if 'P2_Social_Protection' in col else col for col in df_combined.columns]
df_combined.columns = [col.replace('P2_Under5_Mortality', 'sur_u5mor') if 'P2_Under5_Mortality' in col else col for col in df_combined.columns]

In [102]:
df_combined.columns = [col.replace('value_norm', 'norm') if 'value_norm' in col else col for col in df_combined.columns]

In [103]:
# Rename some things
df_combined = df_combined.rename(columns={'name': 'adm_name', 'ISO3':'iso3'})


In [104]:
df_combined = df_combined[[
    'iso3',
    'adm_name',
    'total_pop',
    'u18_pop',
    'wb_income',
    'unicef_ro',
    'ucode',
    'uuid',
    'geometry',
    'type',
    'fragile','rfl_abs',
 'rfl_rel','rfl_abs_norm',
 'rfl_rel_norm','cfl_abs',
 'cfl_rel','cfl_abs_norm',
 'cfl_rel_norm','ts_abs',
 'ts_rel','ts_abs_norm',
 'ts_rel_norm','agdr_abs',
 'agdr_rel','agdr_abs_norm',
 'agdr_rel_norm','metdr_spei_abs',
 'metdr_spei_rel','metdr_spei_abs_norm',
 'metdr_spei_rel_norm','metdr_spi_abs',
 'metdr_spi_rel','metdr_spi_abs_norm',
 'metdr_spi_rel_norm','hw_fre_abs',
 'hw_fre_rel','hw_fre_abs_norm',
 'hw_fre_rel_norm','hw_dur_abs',
 'hw_dur_rel','hw_dur_abs_norm',
 'hw_dur_rel_norm','hw_sev_abs',
 'hw_sev_rel','hw_sev_abs_norm',
 'hw_sev_rel_norm','ext_abs',
 'ext_rel','ext_abs_norm',
 'ext_rel_norm','fr_fre_abs',
 'fr_fre_rel','fr_fre_abs_norm',
 'fr_fre_rel_norm','fr_int_abs',
 'fr_int_rel','fr_int_abs_norm',
 'fr_int_rel_norm','sds_abs',
 'sds_rel','sds_abs_norm',
 'sds_rel_norm','pm25_abs',
 'pm25_rel','pm25_abs_norm',
 'pm25_rel_norm','mal_pv_abs',
 'mal_pv_rel','mal_pv_abs_norm',
 'mal_pv_rel_norm','mal_pf_abs',
 'mal_pf_rel','mal_pf_abs_norm',
 'mal_pf_rel_norm','hea_dtp1','hea_dtp1_norm', 'hea_dtp3','hea_dtp3_norm','hea_skat','hea_skat_norm',
'hea_elec','hea_elec_norm','nut_stu','nut_stu_norm','nut_fpov','nut_fpov_norm','wash_wat',
    'wash_wat_norm','wash_san','wash_san_norm','wash_hyg','wash_hyg_norm','edu_lsos','edu_lsos_norm',
    'edu_lscr','edu_lscr_norm','edu_lpov','edu_lpov_norm','pro_lab','pro_lab_norm','pro_mar',
    'pro_mar_norm','prov_md','prov_md_norm','prov_u5sp','prov_u5sp_norm','sur_u5mor',
    'P1_rfl', 'P1_cfl', 'P1_ts', 'P1_dr', 'P1_hw','P1_ext', 'P1_fr', 'P1_sds', 'P1_pm25', 'P1_mal',
    'P2_hea', 'P2_nut','P2_wash', 'P2_edu', 'P2_pro', 'P2_pov', 'P2_sur','P1_geometric_avg','P2_arithmetic_avg',
'ccri']]

In [105]:
df_combined

,iso3,adm_name,total_pop,u18_pop,wb_income,unicef_ro,ucode,uuid,geometry,type,...,P2_hea,P2_nut,P2_wash,P2_edu,P2_pro,P2_pov,P2_sur,P1_geometric_avg,P2_arithmetic_avg,ccri
0,ALB,Albania,2.785088e+06,574038.05,UMI,ECARO,ALB_V1,b9d8dcda-24d6-45e0-9ccd-170204c795c8,"POLYGON ((19.28752 40.42238, 19.31029 40.3966,...",State,...,0.05,2.70,0.44,0.41,1.97,4.06,0.69,3.28,1.63,4.22
1,AFG,Afghanistan,4.200952e+07,20941093.54,LI,ROSA,AFG_V1,f82a5059-81e7-4d93-8adb-7b1cc1b40a49,"POLYGON ((60.5211 34.10096, 60.52862 34.03797,...",State,...,5.29,6.88,4.62,8.38,4.78,9.96,5.06,5.99,7.25,8.48
2,AND,Andorra,8.141269e+04,12304.82,HI,NaN,AND_V1,0edb2089-1dfc-42cd-a7d8-ccdd7456b358,"POLYGON ((1.44256 42.60367, 1.44601 42.52099, ...",State,...,0.00,NaN,0.00,NaN,NaN,8.78,0.04,1.03,2.46,3.29
3,AGO,Angola,3.670553e+07,18709836.92,LMI,ESARO,AGO_V1,606f8a40-9a97-4198-b75a-7f279b5ce3be,"MULTIPOLYGON (((11.66938 -16.56043, 11.70349 -...",State,...,7.31,6.41,6.88,5.11,4.83,8.50,5.80,5.62,7.23,8.30
4,ARM,Armenia,2.985307e+06,692305.76,UMI,ECARO,ARM_V1,bf63c212-9792-4b8a-9040-75654194c788,"POLYGON ((43.46854 41.0306, 43.49392 41.01885,...",State,...,0.30,1.26,0.44,1.00,0.85,3.48,0.73,2.34,1.26,3.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,ZAF,South Africa,6.285482e+07,19551218.44,UMI,ESARO,ZAF_V1,1724f985-528e-443e-a7a9-4fd3913e7e2a,"POLYGON ((31.30589 -22.41837, 31.27097 -22.396...",State,...,2.38,3.16,3.23,3.53,0.60,2.70,3.06,5.68,2.98,6.20
191,ZWE,Zimbabwe,1.649612e+07,7945012.02,LMI,ESARO,ZWE_V1,2de2cee6-4280-46e6-8032-56c73e8cfc7f,"POLYGON ((25.26144 -17.79225, 25.27674 -17.910...",State,...,2.66,3.97,6.76,4.35,6.31,7.34,3.82,5.12,5.67,7.42
192,ZMB,Zambia,2.095825e+07,10196835.65,LMI,ESARO,ZMB_V1,8228c59a-a030-4ef8-b1d6-f94a04c595d8,"POLYGON ((30.42014 -15.62114, 30.39022 -15.586...",State,...,3.95,4.60,7.26,6.84,4.83,6.45,3.94,4.21,6.10,7.12
193,RUS,Russian Federation,1.444480e+08,29901206.98,HI,NaN,RUS_V1,8e6c566f-9649-4b0f-9727-650d5e073e04,"MULTIPOLYGON (((153.91385 48.92578, 153.95941 ...",State,...,0.14,NaN,0.87,0.08,1.03,0.00,0.22,4.90,0.39,3.00


In [106]:
df_combined[df_combined['iso3']=='MCO']

,iso3,adm_name,total_pop,u18_pop,wb_income,unicef_ro,ucode,uuid,geometry,type,...,P2_hea,P2_nut,P2_wash,P2_edu,P2_pro,P2_pov,P2_sur,P1_geometric_avg,P2_arithmetic_avg,ccri
105,MCO,Monaco,24716.45,5178.62,HI,NaN,MCO_V1,f9260923-3a04-4ec8-95eb-741f3036e821,"POLYGON ((7.4117 43.73428, 7.41831 43.7246, 7....",State,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN


In [107]:
# int vals that are abs
for col in df_combined.columns:
    if 'abs' in col and 'norm' not in col:
        df_combined[col] = df_combined[col].astype(int)

In [108]:
df_combined['total_pop'] = df_combined['total_pop'].round(0)
df_combined['u18_pop'] = df_combined['u18_pop'].round(0)
df_combined['total_pop'] = pd.to_numeric(df_combined['total_pop'], errors='coerce').astype('Int64')
df_combined['u18_pop'] = pd.to_numeric(df_combined['u18_pop'], errors='coerce').astype('Int64')

In [109]:
gdf = gpd.GeoDataFrame(df_combined, geometry=df_combined['geometry'], crs='EPSG:4326')

In [110]:
gdf.to_file('{}/CCRI_results_misc/CCRI_P1_P2_format.geojson'.format(data_dir))